# 03. Phân Tích Hiệu Quả Nhóm Mô Hình Dự Báo

Notebook này đánh giá chi tiết mô hình Random Forest đã được huấn luyện để dự báo nhu cầu.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error

PROJECT_ROOT = Path('../')
MODEL_PATH = PROJECT_ROOT / 'models/rf_forecast_model.pkl'

# Import custom function via sys path hack if needed, or re-implement simple load
import sys
sys.path.append(str(PROJECT_ROOT))
from src.features.timeseries_features import prepare_timeseries_data

print("Setup complete.")

## 1. Tải Dữ Liệu và Mô Hình

In [ ]:
try:
    df = prepare_timeseries_data()
    print(f"Data loaded: {len(df)} rows")
    
    with open(MODEL_PATH, 'rb') as f:
        model = pickle.load(f)
    print("Model loaded successfully.")
except Exception as e:
    print(f"Error: {e}")

## 2. Đánh Giá Trên Tập Huấn Luyện (Training Evaluation)
Xem mô hình khớp với dữ liệu lịch sử như thế nào.

In [ ]:
features = ['lag_1', 'lag_2', 'lag_4', 'rolling_mean_4', 'Month', 'WeekOfYear']
target = 'Quantity'

X = df[features]
y_true = df[target]
y_pred = model.predict(X)

df['Predicted_Qty'] = y_pred
df['Error'] = df['Quantity'] - df['Predicted_Qty']

mae = mean_absolute_error(y_true, y_pred)
print(f"Mean Absolute Error (MAE): {mae:.2f}")

### Biểu đồ Thực tế vs Dự báo (Mẫu 100 điểm)

In [ ]:
sample = df.sample(100)
plt.figure(figsize=(10, 6))
plt.scatter(sample['Quantity'], sample['Predicted_Qty'], alpha=0.6)
plt.plot([sample['Quantity'].min(), sample['Quantity'].max()], 
         [sample['Quantity'].min(), sample['Quantity'].max()], 'r--', lw=2)
plt.xlabel('Thực Tế')
plt.ylabel('Dự Báo')
plt.title('Thực Tế vs Dự Báo (Scatter Plot)')
plt.show()

## 3. Phân Tích Tầm Quan Trọng Của Đặc Trưng (Feature Importance)
Yếu tố nào ảnh hưởng nhất đến nhu cầu?

In [ ]:
importance = pd.DataFrame({
    'Feature': features,
    'Importance': model.feature_importances_
}).sort_values('Importance', ascending=False)

plt.figure(figsize=(10, 5))
sns.barplot(data=importance, x='Importance', y='Feature', palette='viridis')
plt.title('Mức Độ Quan Trọng Của Các Biến Đầu Vào')
plt.show()

**Nhận xét:**
- Thông thường, `lag_1` (tuần trước) và `rolling_mean` có ảnh hưởng lớn nhất vì nhu cầu thường có tính quán tính.
- `WeekOfYear` thể hiện tính mùa vụ.